<a href="https://colab.research.google.com/github/Skapooky/Atividade-Avaliativa/blob/main/TrabalhoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rafaela de Miranda

Guilherme Henrique Peres

Henrique Anderle Schulz

In [ ]:
import os
from pathlib import Path
import numpy as np
import collections
import time
import random
from random import randint

# Abrir o Livro e criar os 100 textos

In [ ]:
livro = '/content/Livro.txt'
pastaOutput = '/content/output'
if not os.path.exists(pastaOutput):
  os.mkdir(pastaOutput)

def LimparOutput(directory):
    directory = Path(directory)
    for item in directory.iterdir():
        if item.is_dir():
            rmdir(item)
        else:
            item.unlink()

LimparOutput(pastaOutput)

with open(livro,'r') as scan:
  data = scan.read()
  start = 0
  end = 300
  lines = data.splitlines()

  for i in range(1, 101):
    ary = np.asarray(lines[start:end])
    arquivo = f"{pastaOutput}/{i}.txt"
    output_file = open(arquivo, "w")
    word = '\n'.join(ary)
    output_file.write(word)
    start = end
    end = end + 300
  
  for i in range(1, 101):
    number_of_words = 0
    output_arquivo = f"{pastaOutput}/{i}.txt"
    with open(output_arquivo,'r') as file:
      data2 = file.read()
      lines2 = data2.split()
      number_of_words += len(lines2)
      
output_file.close()


# Cache LRU

In [ ]:
class Node:
  def __init__(self, key, value):
    self.key = key
    self.value = value
    self.next = None
    self.prev = None

In [ ]:
class CacheLRU:
  def __init__(self):
    self.size = 10
    self.lru_cache = dict()
    self.head = Node(0, 0)
    self.tail = Node(0, 0)
    self.head.next = self.tail
    self.tail.prev = self.head

    self.cacheMiss = 0
    self.cacheHit = 0

  def add(self, node):
    p = self.tail.prev
    p.next = node
    self.tail.prev = node
    node.next = self.tail
    node.prev = p

  def remove(self, node):
    p = node.prev
    n = node.next
    p.next = n
    n.prev = p

  def get(self, key):

    #Encontrou no cache
    if key in self.lru_cache:
      n = self.lru_cache[key]
      self.remove(n)
      self.add(n)
      self.cacheHit += 1
      return n.value[0]
    #Não encontrou no cache
    else:
      with open(f"/content/output/{key}.txt",'r') as scan:
        data = scan.read()
        self.cacheMiss += 1
        saida = self.set(key = key, value = f'{data}')
        return saida

  def set(self, key, value):
    n = Node(key, value)
    self.add(n)
    self.lru_cache[key] = n
    if len(self.lru_cache) > self.size:
      n = self.head.next
      self.remove(n)
      del self.lru_cache[n.key]
    return n.value
 

# CacheLFU

In [ ]:
class CacheLFU:

  def __init__(self):

    self.tabelaHash = dict()
    self.Cache = dict()
    self.contador = 0
    self.cacheHit = 0
    self.cacheMiss = 0

  def freqMin(self, frequencia):
    menorFrequencia = []
    chaves = []
    for x in frequencia:
      menorFrequencia.append(frequencia.get(x))
      chaves.append(x)
    menor = np.min(menorFrequencia)
    pos = menorFrequencia.index(menor)
    return chaves[pos]

  def add(self, chave, conteudo):
    self.Cache[chave] = conteudo
    self.tabelaHash[chave] = 1

  def remove(self):
    menorFreq = self.freqMin(self.tabelaHash)
    del self.tabelaHash[menorFreq]
    del self.Cache[menorFreq]

  def get(self, numeroLivro):
    if numeroLivro not in self.tabelaHash:
      self.cacheMiss += 1
      with open(f"output/{numeroLivro}.txt", "r") as livro:
        conteudo = livro.read()
      if self.contador >= 10:
        self.remove()
      self.add(numeroLivro, conteudo)
      self.contador += 1
      return self.Cache[numeroLivro]
    else:
      self.cacheHit += 1
      self.tabelaHash[numeroLivro] += 1
      return self.Cache[numeroLivro]

# CacheFIFO

In [ ]:
class Lista:
  def __init__(self):
    self.cabeca = None
  
  def __repr__(self):
    return '[' + str(self.cabeca) + ']'

class Nodo:
  def __init__(self, livro, numLivro, proxNodo = None):
    self.livro = livro
    self.proximo = proxNodo
    self.numLivro = numLivro

  def __repr__(self):
        return '%s -> %s' % (self.livro, self.proximo)

class CacheFIFO:
  def __init__(self):
    self.lista = Lista()
    self.cacheMiss = 0
    self.cacheHit = 0
    self.tempo = 0
    self.elementos = 0


  def add(self, lista, livro, numLivro):
    novo_nodo = Nodo(livro, numLivro)
    novo_nodo.proximo = self.lista.cabeca
    self.lista.cabeca = novo_nodo
    return novo_nodo.livro

  def busca(self, lista, numeroLivro):
      livroAtual = self.lista.cabeca
      while livroAtual.numLivro != numeroLivro:
          livroAtual = livroAtual.proximo

  def remove(self, nodo):
    nodo.proximo = None

  def get(self, numeroLivro):
    tempoInicial = time.time()
    livro = CacheFIFO.set(self, int(numeroLivro), self.lista.cabeca, None)
    tempoFinal = time.time()
    self.tempo = (tempoFinal - tempoInicial)
    return livro

  def set(self, numeroLivro, nodo, nodoAnterior):
    if nodo == None:
      self.elementos += 1
      self.cacheMiss += 1
      with open(f"output/{numeroLivro}.txt") as livro:
        conteudo = livro.read()
        livro = CacheFIFO.add(self, self.lista, conteudo, numeroLivro)
        return livro
    elif nodo.proximo == None and nodo.numLivro != numeroLivro:
      self.elementos += 1
      self.cacheMiss += 1
      if self.elementos >= 10:
        CacheFIFO.remove(self, nodoAnterior)
      with open(f"output/{numeroLivro}.txt") as livro:
        conteudo = livro.read()
        livro = CacheFIFO.add(self, self.lista, conteudo, int(numeroLivro))
        return livro
    elif nodo.numLivro == numeroLivro:
      self.cacheHit += 1 
      return nodo.livro
    return CacheFIFO.set(self, int(numeroLivro), nodo.proximo, nodo)

  def percorrer(self, nodo):
    if nodo.proximo != None:
      CacheFIFO.percorrer(self, nodo.proximo)
    else:
      return 1



# Modo Simulação

In [ ]:
 class Simular:
  def __init__(self, cache):
    usuario1 = cache()
    usuario2 = cache()
    usuario3 = cache()
    self.usuarios = [usuario1, usuario2, usuario3]
    self.OutputLog(self.AleatorioPuro())
    self.OutputLog(self.AleatorioPoison())
    self.OutputLog(self.AleatorioManipulado())


  def AleatorioPuro(self):
    log = [self.usuarios[0].__class__.__name__, "Aleatorio Puro"]
    mediaTotal = 0
    mediaMedia = 0
    #Percorre a lista de usuarios
    for index, usuario in enumerate(self.usuarios):
      tempoTotal = 0
      for i in range(200):
        random = randint(1, 100)
        inicio = time.time()
        usuario.get(f"{random}")
        fim = time.time()
        tempoTotal += fim - inicio

      tempoMedio = tempoTotal / 200
      mediaMedia += tempoMedio
      mediaTotal += tempoTotal
      info = [index + 1,
              usuario.cacheHit,
              usuario.cacheMiss,
              tempoTotal,
              tempoMedio
              ]
      log.append(info)
    mediaTotal = mediaTotal / 3
    mediaMedia = mediaMedia / 3

    log.append(mediaTotal)
    log.append(mediaMedia)
    return log
  
  def AleatorioPoison(self):
    parametroPoisson = 5
    log = [self.usuarios[0].__class__.__name__, "Aleatorio Poison"]
    mediaTotal = 0
    mediaMedia = 0


    for index, usuario in enumerate(self.usuarios):
      tempoTotal = 0
      random = np.random.poisson(parametroPoisson, 200) + 1
      for x in random:
        inicio = time.time()
        usuario.get(f"{x}")
        fim = time.time()
        tempoTotal += fim - inicio

      tempoMedio = tempoTotal / 200
      mediaMedia += tempoMedio
      mediaTotal += tempoTotal

      info = [index + 1,
              usuario.cacheHit,
              usuario.cacheMiss,
              tempoTotal,
              tempoMedio
              ]
      log.append(info)
    mediaTotal = mediaTotal / 3
    mediaMedia = mediaMedia / 3
    log.append(mediaTotal)
    log.append(mediaMedia)

    return log

  def AleatorioManipulado(self):
    log = [self.usuarios[0].__class__.__name__, "Aleatorio Manipulado"]
    mediaTotal = 0
    mediaMedia = 0
    for index, usuario in enumerate(self.usuarios):
      tempoTotal = 0
      numeros = np.arange(1, 101, 1)
      weights = []
      for i in range(1, 101):
        if i >= 30  and i <= 40:
          weights.append(3)
        else:
          weights.append(0.75)
      randomNumbers = random.choices(numeros, weights, k=200)
      for x in randomNumbers:
        inicio = time.time()
        usuario.get(f"{x}")
        fim = time.time()
        tempoTotal += fim - inicio

      tempoMedio = tempoTotal / 200
      mediaMedia += tempoMedio
      mediaTotal += tempoTotal

      info = [index + 1,
              usuario.cacheHit,
              usuario.cacheMiss,
              tempoTotal,
              tempoMedio
              ]
      log.append(info)

    mediaTotal = mediaTotal / 3
    mediaMedia = mediaMedia / 3
    log.append(mediaTotal)
    log.append(mediaMedia)
    return log
  
  def OutputLog(self, log):
    cacheLog = open("CacheLog.txt", "a")
    cacheLog.write(f"{log[0]} - ")
    cacheLog.write(f"{log[1]}\n")

    for info in log[2:5]:
      cacheLog.write(f"\tUsuario {info[0]}\n")
      cacheLog.write(f"\t\tCache Hit: {info[1]}\n")
      cacheLog.write(f"\t\tCache Miss: {info[2]}\n")
      cacheLog.write(f"\t\tTempo Total de resposta: {info[3]:10f}\n")
      cacheLog.write(f"\t\tTempo Médio de resposta: {info[4]:10f}\n")

    cacheLog.write(f"\tEficiência do {log[1]} ao carregar 200 arquivos: {log[5]:10f}\n")
    cacheLog.write(f"\tEficiência do {log[1]} ao carregar 1 arquivo: {log[6]:10f}\n")
    cacheLog.write('-'*60 + '\n')

    return

# Main

In [ ]:
tipoCache = int(input("Escolha entre:\n\t[1]Cache LRU\n\t[2]CacheLFU\n\t[3]CacheFIFO\n>>>"))
entradaUsuario = int(input("Digite o número do livro a ser solicitado, ou -1 para entrar no modo de simulação\n\t>>>"))
while(0 > tipoCache > 3 ):
  tipoCache = int(input("Escolha inválida: \n>>>"))

if tipoCache == 1:
  usuario = CacheLRU()
elif tipoCache == 2:
  usuario = CacheLFU()
else:
  usuario = CacheFIFO()

while(entradaUsuario != 0):
  if entradaUsuario == -1:
    cacheLog = open("CacheLog.txt", "w")
    cacheLog.write('')
    Simular(CacheLRU)
    Simular(CacheFIFO)
    Simular(CacheLFU)
    entradaUsuario = int(input("Digite o número do livro a ser solicitado, ou -1 para entrar no modo de simulação\n\t>>>"))
  else:
    fim = 0
    inicio = time.time()
    saida = usuario.get(entradaUsuario)
    fim = time.time()
    print(saida)
    tempo = fim - inicio
    print(f"Usuario:\n\tCache Hit: {usuario.cacheHit}\n\tCache Miss: {usuario.cacheMiss}\n\tTempo de resposta: {tempo:.20f}")
    entradaUsuario = int(input("Digite o número do texto a ser solicitado, ou -1 para entrar no modo de simulação\n\t>>>"))

KeyboardInterrupt: ignored